In [1]:
import numpy as np
import pandas as pd
import os
import quandl as ql

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
QUANDL_KEY = os.getenv("QUANDL_KEY")

In [4]:
ql.ApiConfig.api_key = QUANDL_KEY

In [117]:
def cleanData(df, asset_class, security):
    cleaned_df = df[['Noncommercial Long', 'Noncommercial Short']]
    cleaned_df['net_noncomercial'] = cleaned_df['Noncommercial Long'] - cleaned_df['Noncommercial Short']
    cleaned_df['asset_class'] = asset_class
    cleaned_df['security'] = security
    return cleaned_df[['asset_class','security','net_noncomercial']]

In [118]:
def get_stats(df, window):
    roling_df = df.rolling(window=window)
    mean = roling_df.mean().shift(1)
    std = roling_df.std(ddof=0).shift(1)
    z_score = (df-mean)/std
    return mean, std, z_score

In [143]:
def get_stats_multi_interval(df, windows):
    df_temp = pd.DataFrame()
    for window in windows: 
        roling_df = df.rolling(window=window)
        mean = roling_df.mean().shift(1)
        std = roling_df.std(ddof=0).shift(1)
        z_score = (df-mean)/std
        df_temp[f"mean_{window//57} years"] = mean
        df_temp[f"std_{window//57} years"] = std
        df_temp[f"z_score_{window//57} years"] = z_score
    return df_temp

In [144]:
def createSummaryDF(df, interval):
    #df = df.reset_index()
    summary_df = pd.DataFrame()
    for i in range(len(df.index)):
        if i > interval and i % interval == 0:
            summary_df = summary_df.append(df.iloc[i])
    return summary_df

In [150]:
data = ql.get('CFTC/084691_FO_L_ALL')
interval = 52*3
data_cleaned = cleanData(data, "Commodity", 'Silver')
stats_df = get_stats_multi_interval(data_cleaned.loc[:,'net_noncomercial'],[57*3,57*5,57*10])
df_concate = pd.concat([data_cleaned,stats_df], join='inner', axis=1)
summary_df = df_concate.last('1d')
summary_df

C:\Users\sinth\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sinth\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\sinth\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,asset_class,security,net_noncomercial,mean_3 years,std_3 years,z_score_3 years,mean_5 years,std_5 years,z_score_5 years,mean_10 years,std_10 years,z_score_10 years
Date,,,,,,,,,,,,
2020-07-14,Commodity,Silver,43348.0,31516.654971,29995.960843,0.394431,39700.424561,30407.65898,0.119956,33113.691228,24501.830992,0.417696


0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
5            NaN
6            NaN
7            NaN
8            NaN
9            NaN
10           NaN
11           NaN
12           NaN
13           NaN
14           NaN
15           NaN
16           NaN
17           NaN
18           NaN
19           NaN
20           NaN
21           NaN
22           NaN
23           NaN
24           NaN
25           NaN
26           NaN
27           NaN
28           NaN
29           NaN
          ...   
1291    0.976593
1292    1.179668
1293    1.105988
1294    1.069950
1295    1.116454
1296    0.905019
1297    1.069745
1298    1.054140
1299    1.452865
1300    1.281894
1301    0.395215
1302    0.340840
1303    0.111170
1304   -0.111851
1305   -0.101784
1306   -0.114042
1307   -0.070415
1308   -0.166777
1309   -0.165518
1310   -0.268039
1311   -0.164133
1312    0.075246
1313    0.259001
1314    0.282263
1315    0.148331
1316    0.270727
1317    0.314109
1318    0.3275

Int64Index([21, 28,  4, 18, 25,  2,  9, 16, 23, 30,
            ...
            12, 19, 26,  2,  9, 16, 23, 30,  7, 14],
           dtype='int64', name='Date', length=1321)

DatetimeIndex(['1995-03-21', '1995-03-28', '1995-04-04', '1995-04-18',
               '1995-04-25', '1995-05-02', '1995-05-09', '1995-05-16',
               '1995-05-23', '1995-05-30',
               ...
               '2020-05-12', '2020-05-19', '2020-05-26', '2020-06-02',
               '2020-06-09', '2020-06-16', '2020-06-23', '2020-06-30',
               '2020-07-07', '2020-07-14'],
              dtype='datetime64[ns]', name='Date', length=1321, freq=None)